<a href="https://colab.research.google.com/github/edgarlei/2020-21-Concrete-Canoe/blob/main/Paper_Replication_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up

In [37]:
import pandas as pd

In [38]:
# Importing CSV file
rep = pd.read_csv('Raw Data Master Sheet.csv')

In [39]:
rep

,Slag (kg),Fly ash (kg),Coarse Aggregates (kg),Fine Aggregates (kg),NaOH Activator (kg),Na-Silicate Activator (kg),Solid % (Na-Silicate) Added Water (kg),Solid % (Na-Silicate) SiO2,Solid % (Na-Silicate) Na2O,NaOH Solution Molarity (M),NaOH Solution Density (g/cm^3),"Compressive strength, Mpa",Unnamed: 12
0,240,80,1215,715,46,114,80,29.4,14.7,12,1.37,35.5,NaN
1,240,80,1215,715,46,114,80,29.4,14.7,12,1.37,47.7,NaN
2,340,85,1065,390,164,0,58,29.4,14.7,12,1.37,37.0,NaN
3,340,85,1065,390,82,82,67,29.4,14.7,12,1.37,56.0,NaN
4,223,95,1160,753,61,75,85,25.5,12.8,10,1.32,64.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,41,367,1294,554,41,103,0,28.0,8.0,8,1.27,21.1,NaN
239,165,385,913,508,97,244,0,29.4,14.7,12,1.37,43.3,NaN
240,100,400,805,925,50,125,12,29.4,14.7,12,1.37,34.9,NaN
241,45,405,1000,850,57,143,0,29.4,13.7,12,1.37,35.9,NaN


# Equations

In [15]:
# Na-Silicate Solution
nas_s = rep['Solid % (Na-Silicate) SiO2'] + rep['Solid % (Na-Silicate) Na2O']

# H2O Present in Na-Silicate Solution
h2o_nas = rep['Na-Silicate Activator (kg)'] - (nas_s)

# NaOH Solution
def naoh_solution():
  numer1 = 40 * rep['NaOH Solution Molarity (M)']
  denom1 = 1000 * rep['NaOH Solution Density (g/cm^3)']
  return rep['NaOH Activator (kg)'] * (numer1/denom1)

naoh_s = naoh_solution()

# H2O Present in NaOH Solution
h2o_naoh = rep['NaOH Activator (kg)'] - naoh_s

In [16]:
# Putting Equations Results into Table
eqns_table = pd.DataFrame({"Na-Silicate Solution": nas_s, "H2O Present in Na-Silicate Solution": h2o_nas, "NaOH Solution": naoh_s, "H2O Present in NaOH Solution": h2o_naoh})
eqns_table

,Na-Silicate Solution,H2O Present in Na-Silicate Solution,NaOH Solution,H2O Present in NaOH Solution
0,44.1,69.9,16.116788,29.883212
1,44.1,69.9,16.116788,29.883212
2,44.1,-44.1,57.459854,106.540146
3,44.1,37.9,28.729927,53.270073
4,38.3,36.7,18.484848,42.515152
...,...,...,...,...
238,36.0,67.0,10.330709,30.669291
239,44.1,199.9,33.985401,63.014599
240,44.1,80.9,17.518248,32.481752
241,43.1,99.9,19.970803,37.029197


# 5 (+1) Key Parameters

In [17]:
# Fly ash/Slag Ratio
flyash_slag = rep['Fly ash (kg)'] / rep['Slag (kg)']

# Water/Solid Ratio
def water_solid_ratio():
  numer2 = rep['Solid % (Na-Silicate) Added Water (kg)'] + h2o_nas + h2o_naoh
  denom2 = rep['Slag (kg)'] + rep['Fly ash (kg)'] + nas_s + naoh_s
  return numer2/denom2

water_solid = water_solid_ratio()

# Activator/Binder Ratio
def act_bin_ratio():
  numer3 = rep['NaOH Activator (kg)'] + rep['Na-Silicate Activator (kg)']
  denom3 = rep['Slag (kg)'] + rep['Fly ash (kg)']
  return numer3/denom3

act_bin = act_bin_ratio()

# Na-Silicate/NaOH Ratio
nasil_naoh = rep['Na-Silicate Activator (kg)'] / rep['NaOH Activator (kg)']

# NaOH Solution Molarity
naoh_mol = rep['NaOH Solution Molarity (M)']

# Compressive Strength
comp_str = rep['Compressive strength, Mpa']

In [18]:
# Key Parameters Table
key_pars = pd.DataFrame({"Fly ash/Slag Ratio": flyash_slag, "Water/Solid Ratio": water_solid, "Activator/Binder Ratio": act_bin, "Na-Silicate/NaOH Ratio": nasil_naoh, "NaOH Solution Molarity": naoh_mol, "Compressive Strength": comp_str})
key_pars

,Fly ash/Slag Ratio,Water/Solid Ratio,Activator/Binder Ratio,Na-Silicate/NaOH Ratio,NaOH Solution Molarity,Compressive Strength
0,0.333333,0.472844,0.500000,2.478261,12,35.5
1,0.333333,0.472844,0.500000,2.478261,12,47.7
2,0.250000,0.228730,0.385882,0.000000,12,37.0
3,0.250000,0.317719,0.385882,1.000000,12,56.0
4,0.426009,0.438158,0.427673,1.229508,10,64.3
...,...,...,...,...,...,...
238,8.951220,0.214974,0.352941,2.512195,8,21.1
239,2.333333,0.418597,0.620000,2.515464,12,43.3
240,4.000000,0.223251,0.350000,2.500000,12,34.9
241,9.000000,0.266882,0.444444,2.508772,12,35.9


In [29]:
# Converting to Array
key_pars_array = key_pars.to_numpy()

# Attempt at Hampel Method
Code is basically verbatim from https://www.youtube.com/watch?v=lHBKIqxxc1Q&ab_channel=TalhaIqbal

I haven't made much successful headway in figuring out how it works and getting a result.

In [19]:
import numpy as np

In [33]:
def med_abs_dev(datapoints):
  med2 = np.median(datapoints)
  return np.median(np.abs(datapoints - med2))

In [36]:
def hampel_method(raw_signal, win_size=3, sigma=3):
  copy_signal = np.copy(np.asarray(raw_signal))
  n = len(raw_signal)
  for i in range((win_size), (n - win_size)):
    dataslice = copy_signal[i - win_size : i + win_size]
    median_abs_dev = med_abs_dev(dataslice)
    med = np.median(dataslice)
    if copy_signal[i] > med + (sigma * median_abs_dev):
      copy_signal[i] = med
    return copy_signal

hampel_method(key_pars_array)

ValueError: ignored